# huntabyte/shadcn-svelte

> 

In [ ]:
from lovely_docs.settings import settings, GitSource

from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import logging
import subprocess

from lovely_docs.git import clone_repo

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
build = True

source = GitSource(
    name="huntabyte/shadcn-svelte",
    doc_dir="docs/.svelte-kit/cloudflare/docs",
    repo="https://github.com/huntabyte/shadcn-svelte",
    commit="main",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

64: 157.0/157.0 completed with 4 local objects.

In [ ]:
commit, clone_dir

('2c94bbbe096fcebb49282369b8a91a7e2e73eb75',
 Path('../git_dir/github.com/huntabyte/shadcn-svelte'))

In [ ]:
if build:
    with open("huntabyte-shadcn-svelte.log", "w") as log_file:
        # Clean the repo
        subprocess.run(["git", "clean", "-dfx"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in root
        subprocess.run(["pnpm", "install"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Build the package and docs
        subprocess.run(["pnpm", "build"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # # Install dependencies in docs
        # subprocess.run(["pnpm", "install"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

        # # Build the docs
        # subprocess.run(["pnpm", "build"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

In [ ]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)

In [ ]:
# Path("shadcn-svelte.json").write_text(tree.model_dump_json(indent=2))

In [ ]:
# tree.subdirs = tree.subdirs[:3]
# tree.subdirs[1].subdirs = tree.subdirs[1].subdirs[:4]

#### Split the single llms.txt file into pages and directories based on heading levels

In [ ]:
# compacted_tree = compact_tree(tree)

In [ ]:
# Path("shadcn-svelte-compact.json").write_text(compacted_tree.model_dump_json(indent=2))

In [ ]:
extra = """In cli examples with multipls pms, only keep one for npm.
For component install, add -y -o to the example with explanation:
  -y: skip confirmation prompt (default: false)
  -o: overwrite existing files (default: false)
`npx shadcn-svelte@latest add <component> -y -o`
"""

processed_tree = await process_tree_depth_first(settings, tree, source.name, extra, extra)

+168.305s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/accordion.md
+0.244s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/alert-dialog.md
+0.017s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/alert.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/aspect-ratio.md
+0.016s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/avatar.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/badge.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/breadcrumb.md
+0.017s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/button-group.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/button.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/calendar.md
+0.015s DEBUG lovely_docs/docs.py:153 llm_process_page Processing components/card.md
+0.015s DEBUG love

In [ ]:
# Path("shadcn-svelte-processed.json").write_text(processed_tree.model_dump_json(indent=2))

#### Calculate cost

In [ ]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 381,758
  Output tokens: 123,436
  Total tokens: 505,194
Cost:
  Total: $1.00
  Input: $0.38
  Output: $0.62


#### save the results

In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)

In [ ]:
processed_tree.children

[DocItem(origPath=Path('components'), name='ui_components', displayName='UI Components', digest='A comprehensive collection of 60+ reusable Svelte UI components built on Bits UI, Embla Carousel, LayerChart, and other libraries. Each component is installed via `npx shadcn-svelte@latest add <component> -y -o` (flags: -y skips confirmation, -o overwrites existing files).\n\n**Core Components:**\n- **Accordion**: Vertically stacked expandable sections with single/multiple modes, WAI-ARIA accessible\n- **Alert/AlertDialog**: Callout notifications and modal dialogs with variants (default, destructive)\n- **Avatar**: Image with fallback text, composed of Root/Image/Fallback\n- **Badge**: Styled badges with variants (default, secondary, destructive, outline), icon support\n- **Breadcrumb**: Navigation path with Root/List/Item/Link/Page/Separator/Ellipsis, custom separators, dropdowns, responsive variants\n- **Button**: Reusable button with variants (default, secondary, destructive, outline, gh